In [70]:
import datasets
cache_path = '/data/kcl/lpy/hf'
data_path = "/data/kcl/lpy/data/AIME_2024"
dataset = datasets.load_dataset(data_path, "default", cache_dir=cache_path)

In [72]:
    train_dataset = dataset["train"]

    # add a row to each data item that represents a unique id
    def make_map_fn(split):
        def process_fn(example, idx):
            orig_extra_info = example.pop("extra_info")
            extra_info = orig_extra_info.copy()
            extra_info["need_tools_kwargs"] = True
            extra_info["tools_kwargs"] = {
                "code_interpreter": {
                    "create_kwargs": {
                        "ground_truth": example["reward_model"]["ground_truth"],
                    },
                },
            }
            example["extra_info"] = extra_info
            example['data_source'] = 'aime_2024'
            return example

        return process_fn

    train_dataset = train_dataset.map(function=make_map_fn("train"), with_indices=True).remove_columns("__index_level_0__")
    train_dataset = datasets.concatenate_datasets([train_dataset, train_dataset]).shuffle(seed=7)

Map: 100%|██████████| 960/960 [00:00<00:00, 10316.35 examples/s]


In [75]:
    local_dir = '/data/kcl/lpy/data/AIME_2024/'
    import os
    train_dataset.to_parquet(os.path.join(local_dir, "test_verl.parquet"))

Creating parquet from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 11.38ba/s]


1860608

In [18]:
__index_level_0__list = []
for item in train_dataset:
    __index_level_0__list.append(item['__index_level_0__'])

In [23]:
len(set(__index_level_0__list))

960

In [21]:
len(train_dataset)

960

In [13]:
datasets.__version__

'4.5.0'